In [12]:
import nltk
nltk.download('stopwords')
from nltk.corpus import *
import pandas as pd
import numpy as np
import os
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shvmt\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [15]:
def load_preprocess(path):
    os.listdir(path)
    filenames = os.listdir(path)
    content = []
    for i in range(len(filenames)):
        f = open(path + filenames[i])
        content.append(f.read())
        f.close()
    normalize_tokenize_nostop=[]
    stopword = nltk.corpus.stopwords.words('english')
    for i in content:
        normalize_tokenize_nostop.append([word.lower() for word in i.split() if word.isalpha() and word not in stopword])
    ps = PorterStemmer()
    final = []
    for i in normalize_tokenize_nostop:
        final.append(' '.join([ps.stem(word) for word in i]))
    return final

In [16]:
path = 'C:/Users/shvmt/Documents/GitHub/nlp/dataset/'
final = load_preprocess(path)
final

['data scienc import field scienc',
 'thi import data scienc cours',
 'the car driven road',
 'the truck driven highway']

In [21]:
filenames = os.listdir(path)
doc_count = {}
for i in range(len(final)):
    word_count = {}
    for word in final[i].split():
        if word not in word_count.keys():
            word_count[word] = 0
        word_count[word] += 1
    doc_count[filenames[i]] = word_count
doc_count

{'1.txt': {'data': 1, 'scienc': 2, 'import': 1, 'field': 1},
 '2.txt': {'thi': 1, 'import': 1, 'data': 1, 'scienc': 1, 'cours': 1},
 '3.txt': {'the': 1, 'car': 1, 'driven': 1, 'road': 1},
 '4.txt': {'the': 1, 'truck': 1, 'driven': 1, 'highway': 1}}

In [66]:
tdm_tf_scratch = pd.DataFrame(doc_count)
tdm_tf_scratch.fillna(0, inplace=True)
tdm_tf_scratch

,1.txt,2.txt,3.txt,4.txt
data,1.0,1.0,0.0,0.0
scienc,2.0,1.0,0.0,0.0
import,1.0,1.0,0.0,0.0
field,1.0,0.0,0.0,0.0
thi,0.0,1.0,0.0,0.0
cours,0.0,1.0,0.0,0.0
the,0.0,0.0,1.0,1.0
car,0.0,0.0,1.0,0.0
driven,0.0,0.0,1.0,1.0
road,0.0,0.0,1.0,0.0


In [67]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x = cv.fit_transform(final)

In [68]:
print(x)

  (0, 2)	1
  (0, 8)	2
  (0, 6)	1
  (0, 4)	1
  (1, 2)	1
  (1, 8)	1
  (1, 6)	1
  (1, 10)	1
  (1, 1)	1
  (2, 9)	1
  (2, 0)	1
  (2, 3)	1
  (2, 7)	1
  (3, 9)	1
  (3, 3)	1
  (3, 11)	1
  (3, 5)	1


In [69]:
tdm_tf_inbuilt = pd.DataFrame(x.toarray().T, index=cv.get_feature_names()) #rows are terms, columns are documents
tdm_tf_inbuilt

,0,1,2,3
car,0,0,1,0
cours,0,1,0,0
data,1,1,0,0
driven,0,0,1,1
field,1,0,0,0
highway,0,0,0,1
import,1,1,0,0
road,0,0,1,0
scienc,2,1,0,0
the,0,0,1,1


In [70]:
tdm_len_norm_scratch = tdm_tf_scratch.iloc[:,:].div(tdm_tf_scrath.sum(axis=0),axis=1) #all rows for each column, that is axis = 0

In [71]:
tdm_len_norm_scratch

,1.txt,2.txt,3.txt,4.txt
data,0.2,0.2,0.00,0.00
scienc,0.4,0.2,0.00,0.00
import,0.2,0.2,0.00,0.00
field,0.2,0.0,0.00,0.00
thi,0.0,0.2,0.00,0.00
cours,0.0,0.2,0.00,0.00
the,0.0,0.0,0.25,0.25
car,0.0,0.0,0.25,0.00
driven,0.0,0.0,0.25,0.25
road,0.0,0.0,0.25,0.00


In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(norm='l1', use_idf=False)
Y = tf.fit_transform(final)
print(Y)

  (0, 2)	0.2
  (0, 8)	0.4
  (0, 6)	0.2
  (0, 4)	0.2
  (1, 2)	0.2
  (1, 8)	0.2
  (1, 6)	0.2
  (1, 10)	0.2
  (1, 1)	0.2
  (2, 9)	0.25
  (2, 0)	0.25
  (2, 3)	0.25
  (2, 7)	0.25
  (3, 9)	0.25
  (3, 3)	0.25
  (3, 11)	0.25
  (3, 5)	0.25


In [73]:
tdm_len_norm_inbuilt = pd.DataFrame(Y.toarray().T, index=cv.get_feature_names())
tdm_len_norm_inbuilt

,0,1,2,3
car,0.0,0.0,0.25,0.00
cours,0.0,0.2,0.00,0.00
data,0.2,0.2,0.00,0.00
driven,0.0,0.0,0.25,0.25
field,0.2,0.0,0.00,0.00
highway,0.0,0.0,0.00,0.25
import,0.2,0.2,0.00,0.00
road,0.0,0.0,0.25,0.00
scienc,0.4,0.2,0.00,0.00
the,0.0,0.0,0.25,0.25


In [74]:
tdm_max_norm_scrath = tdm_tf_scrath.iloc[:,:].div(tdm_tf_scrath.max(axis=0),axis=1)
tdm_max_norm_scrath 
# instead of sum, it is .max()

,1.txt,2.txt,3.txt,4.txt
data,0.5,1.0,0.0,0.0
scienc,1.0,1.0,0.0,0.0
import,0.5,1.0,0.0,0.0
field,0.5,0.0,0.0,0.0
thi,0.0,1.0,0.0,0.0
cours,0.0,1.0,0.0,0.0
the,0.0,0.0,1.0,1.0
car,0.0,0.0,1.0,0.0
driven,0.0,0.0,1.0,1.0
road,0.0,0.0,1.0,0.0


In [75]:
tdm_log_norm_scrath = np.log10(1+tdm_max_norm_scrath) #we add 1 since, many terms are 0
tdm_log_norm_scrath

,1.txt,2.txt,3.txt,4.txt
data,0.176091,0.30103,0.00000,0.00000
scienc,0.301030,0.30103,0.00000,0.00000
import,0.176091,0.30103,0.00000,0.00000
field,0.176091,0.00000,0.00000,0.00000
thi,0.000000,0.30103,0.00000,0.00000
cours,0.000000,0.30103,0.00000,0.00000
the,0.000000,0.00000,0.30103,0.30103
car,0.000000,0.00000,0.30103,0.00000
driven,0.000000,0.00000,0.30103,0.30103
road,0.000000,0.00000,0.30103,0.00000


In [78]:
idf_count = {}
unique_terms = tdm_tf_scratch.index
for term in unique_terms:
    idf_count[term]=0
    for doc in final:
        if term in doc.split():
            idf_count[term] += 1
    idf_count[term] = np.log10(len(final)/idf_count[term])
idf_count

{'data': 0.3010299956639812,
 'scienc': 0.3010299956639812,
 'import': 0.3010299956639812,
 'field': 0.6020599913279624,
 'thi': 0.6020599913279624,
 'cours': 0.6020599913279624,
 'the': 0.3010299956639812,
 'car': 0.6020599913279624,
 'driven': 0.3010299956639812,
 'road': 0.6020599913279624,
 'truck': 0.6020599913279624,
 'highway': 0.6020599913279624}

In [79]:
idf_Series = pd.Series(idf_count)
idf = pd.DataFrame(idf_Series)
idf

,0
data,0.30103
scienc,0.30103
import,0.30103
field,0.60206
thi,0.60206
cours,0.60206
the,0.30103
car,0.60206
driven,0.30103
road,0.60206


In [81]:
tfidf_scratch = tdm_len_norm_scratch*idf.values
tfidf_scratch

,1.txt,2.txt,3.txt,4.txt
data,0.060206,0.060206,0.000000,0.000000
scienc,0.120412,0.060206,0.000000,0.000000
import,0.060206,0.060206,0.000000,0.000000
field,0.120412,0.000000,0.000000,0.000000
thi,0.000000,0.120412,0.000000,0.000000
cours,0.000000,0.120412,0.000000,0.000000
the,0.000000,0.000000,0.075257,0.075257
car,0.000000,0.000000,0.150515,0.000000
driven,0.000000,0.000000,0.075257,0.075257
road,0.000000,0.000000,0.150515,0.000000


In [84]:
tf1 = TfidfVectorizer(norm='l1')
z = tf1.fit_transform(final)
tfidf_inbuilt = pd.DataFrame(z.toarray().T, index=tf1.get_feature_names(), columns=filenames)
tfidf_inbuilt

,1.txt,2.txt,3.txt,4.txt
car,0.000000,0.000000,0.555283,0.000000
cours,0.000000,0.508672,0.000000,0.000000
data,0.362529,0.401043,0.000000,0.000000
driven,0.000000,0.000000,0.437791,0.437791
field,0.459822,0.000000,0.000000,0.000000
highway,0.000000,0.000000,0.000000,0.555283
import,0.362529,0.401043,0.000000,0.000000
road,0.000000,0.000000,0.555283,0.000000
scienc,0.725058,0.401043,0.000000,0.000000
the,0.000000,0.000000,0.437791,0.437791
